In [ ]:
# 1. Proje Kurulumu ve Güncelleme
import os
import sys

# Google Colab kontrolü
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Google Colab ortamı algılandı.")
    repo_url = "https://github.com/onndd/newproje.git"
    project_dir = "/content/newproje"
    
    if os.path.exists(project_dir):
        print("Proje klasörü mevcut. Güncelleniyor...")
        %cd {project_dir}
        !git pull
    else:
        print("Proje klonlanıyor...")
        !git clone {repo_url}
        %cd {project_dir}
        
    # Gerekli kütüphaneler (Optuna eklendi)
    print("Kütüphaneler yükleniyor...")
    %pip install catboost hmmlearn tensorflow lightgbm scikit-learn optuna
    
    # Path ayarı
    sys.path.append(project_dir)
else:
    print("Yerel ortam algılandı.")
    project_dir = os.getcwd()
    sys.path.append(project_dir)

In [ ]:
# 2. Veri Yükleme
import sqlite3
import pandas as pd
import numpy as np
from jetx_project.data_loader import load_data
from jetx_project.config import DB_PATH

print("Veri yükleniyor...")
try:
    df = load_data(DB_PATH)
    print(f"Toplam {len(df)} veri yüklendi.")
    print(df.head())
except Exception as e:
    print(f"Hata: {e}")
    print("Lütfen jetx.db dosyasının proje klasöründe olduğundan emin olun.")

In [ ]:
# 3. Psikolojik Modeller (HMM) Eğitimi - SIZINTISIZ (LEAK-FREE)
print("Psikolojik Modeller (HMM) eğitiliyor...")
from jetx_project.model_hmm import train_hmm_model, predict_hmm_state, save_hmm_model

if 'df' in locals() and len(df) > 0:
    # --- HMM VERİ SIZINTISI (DATA LEAKAGE) DÜZELTMESİ ---
    # HMM'i sadece eğitim verisiyle eğitmeliyiz.
    # Test verisini "görmemeli".
    
    # 1. Veriyi Böl (Zaman Serisi Olduğu İçin Karıştırmadan)
    train_size = int(len(df) * 0.85)
    train_values = df['value'].values[:train_size]
    test_values = df['value'].values[train_size:]
    
    print(f"HMM Eğitimi için veri ayrıldı: {len(train_values)} örnek (Toplamın %85'i)")
    
    # 2. HMM Eğitimi (Sadece Train Seti)
    # Log-Transform kullanılarak eğitiliyor (model_hmm.py içinde güncellendi)
    hmm_model, hmm_map = train_hmm_model(train_values)
    save_hmm_model(hmm_model, hmm_map, output_dir='models')
    
    # 3. Durum Tahmini (Forward-Only)
    # Train seti için tahmin (Model zaten bunu gördü, sorun yok)
    train_states = predict_hmm_state(hmm_model, train_values, hmm_map)
    
    # Test seti için tahmin (Model bunu GÖRMEDİ, sadece öğrendiği parametreleri kullanıyor)
    # predict_hmm_state fonksiyonu modeli yeniden eğitmez, sadece tahmin yapar.
    test_states = predict_hmm_state(hmm_model, test_values, hmm_map)
    
    # 4. Birleştirme (Feature Extraction için)
    # Artık elimizde tüm veri seti için durumlar var, ama test kısmındaki durumlar
    # sadece geçmiş bilgiyle (train parametreleriyle) üretildi.
    hmm_states_mapped = np.concatenate([train_states, test_states])
    
    print("HMM Durumları belirlendi.")
    print(f"Durum Dağılımı: {np.bincount(hmm_states_mapped)}")
else:
    print("Veri yok, HMM eğitilemedi.")

In [ ]:
# 4. OPTUNA İLE HİPERPARAMETRE OPTİMİZASYONU
from jetx_project.optimization import optimize_catboost, optimize_lightgbm
from jetx_project.model_a import prepare_model_a_data

if 'df' in locals() and len(df) > 500:
    print("Optimizasyon için veri hazırlanıyor...")
    X, y_p15, y_p3, y_x = prepare_model_a_data(df['value'].values, hmm_states_mapped)
    
    # Sadece eğitim verisi üzerinde optimizasyon yapalım
    split_idx = int(len(X) * 0.85)
    X_train_opt = X.iloc[:split_idx]
    y_p15_train_opt = y_p15[:split_idx]
    
    print("--- CatBoost Optimizasyonu Başlıyor (GPU) ---")
    # P1.5 için en iyi parametreleri bulalım
    best_params_catboost = optimize_catboost(X_train_opt, y_p15_train_opt, n_trials=20)
    print("Bulunan En İyi Parametreler:", best_params_catboost)
    
    # Not: LightGBM optimizasyonu CPU üzerinde çalışır, isteğe bağlı açılabilir.
    # best_params_lgb = optimize_lightgbm(X_train_opt, y_p15_train_opt, n_trials=10)
else:
    print("Yeterli veri yok.")

In [ ]:
# 5. Model A (CatBoost) Eğitimi
from jetx_project.model_a import train_model_a, save_models
from catboost import CatBoostClassifier

if 'df' in locals() and len(df) > 500:
    print("Model A (CatBoost) eğitiliyor...")
    # train_model_a fonksiyonu şu an sabit parametreler kullanıyor.
    # Ancak biz optimize edilmiş parametreleri kullanmak istiyoruz.
    # Bu yüzden burada manuel eğitim yapabiliriz veya train_model_a'yı güncelleyebiliriz.
    # Şimdilik standart eğitimi çalıştıralım, ama ileride parametreleri enjekte edebiliriz.
    # (Not: optimize_catboost zaten en iyi parametreleri yazdırdı, manuel olarak koda eklenebilir)
    
    # TODO: best_params_catboost'u train_model_a'ya geçirmek için kodu güncellemek gerekebilir.
    # Şimdilik varsayılan (iyileştirilmiş) ayarlarla devam ediyoruz.
    modelA_p15, modelA_p3, modelA_x = train_model_a(X, y_p15, y_p3, y_x)
    save_models(modelA_p15, modelA_p3, modelA_x, output_dir='models')
    print("Model A tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 6. Model B (k-NN) Eğitimi - OPTİMİZE EDİLMİŞ (PCA)
from jetx_project.model_b import build_memory, train_model_b, save_memory

if 'df' in locals() and len(df) > 500:
    print("Model B (Hafıza) oluşturuluyor...")
    patterns, targets = build_memory(df['value'].values)
    
    # Artık PCA nesnesini de döndürüyor
    nbrs, pca = train_model_b(patterns)
    
    save_memory(nbrs, pca, patterns, targets, output_dir='models')
    print("Model B tamamlandı (PCA ile optimize edildi).")
else:
    print("Yeterli veri yok.")

In [ ]:
# 7. Model C (LSTM) Eğitimi - SIZINTISIZ
from jetx_project.model_lstm import train_model_lstm, save_lstm_models

if 'df' in locals() and len(df) > 500:
    print("Model C (LSTM) eğitiliyor...")
    # LSTM için veriyi kendi içinde hazırlar (Artık kronolojik split kullanıyor)
    modelC_p15, modelC_p3, scaler_lstm = train_model_lstm(df['value'].values)
    save_lstm_models(modelC_p15, modelC_p3, scaler_lstm, output_dir='models')
    print("Model C tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 8. Model D (LightGBM) Eğitimi
from jetx_project.model_lightgbm import train_model_lightgbm, save_lightgbm_models

if 'df' in locals() and len(df) > 500:
    print("Model D (LightGBM) eğitiliyor...")
    # Model A ile aynı feature setini kullanır
    modelD_p15, modelD_p3 = train_model_lightgbm(X, y_p15, y_p3)
    save_lightgbm_models(modelD_p15, modelD_p3, output_dir='models')
    print("Model D tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 9. Model E (MLP) Eğitimi
from jetx_project.model_mlp import train_model_mlp, save_mlp_models

if 'df' in locals() and len(df) > 500:
    print("Model E (MLP) eğitiliyor...")
    # Model A ile aynı feature setini kullanır ama içinde filtreler
    modelE_p15, modelE_p3, mlp_cols = train_model_mlp(X, y_p15, y_p3)
    save_mlp_models(modelE_p15, modelE_p3, mlp_cols, output_dir='models')
    print("Model E tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 9.5. ENSEMBLE STACKING (Meta-Learner) Eğitimi
from jetx_project.ensemble import prepare_meta_features, train_meta_learner, save_meta_learner
from jetx_project.model_lstm import create_sequences

if 'df' in locals() and len(df) > 2000:
    print("--- Ensemble Stacking (Meta-Learner) Eğitiliyor ---")
    
    # Meta-Learner için VALIDATION setini kullanmalıyız.
    # Train setini kullanırsak overfitting olur.
    # Bu yüzden son %15'lik kısmı (Validation) kullanacağız.
    
    val_size = int(len(df) * 0.15)
    val_start_idx = len(df) - val_size
    
    # 1. Validation Seti İçin Tahminleri Topla
    print("Modellerin Validation tahminleri toplanıyor...")
    
    # Model A (CatBoost)
    X_val_a, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=val_start_idx)
    # Hizalama: prepare_model_a_data N-1 döndürür
    # Hedeflerimiz de buna uymalı
    y_val_true_15 = (df['value'].values[val_start_idx+1:] >= 1.5).astype(int)
    
    preds_a_val = modelA_p15.predict_proba(X_val_a)[:, 1]
    
    # Model B (k-NN)
    preds_b_val = []
    from jetx_project.model_b import create_pattern_vector, predict_model_b
    
    # PCA nesnesini Cell 6'dan alıyoruz
    pca_obj = locals().get('pca', None) 
    
    for i in range(val_start_idx, len(df) - 1):
        pat = create_pattern_vector(df['value'].values, i)
        if pat is not None:
            p15, _, _ = predict_model_b(nbrs, pca_obj, targets, pat)
            preds_b_val.append(p15)
        else:
            preds_b_val.append(0.5)
    preds_b_val = np.array(preds_b_val)
            
    # Model C (LSTM)
    seq_len = 200
    val_lstm_start = val_start_idx - seq_len
    val_values_extended = df['value'].values[val_lstm_start:]
    val_values_scaled = scaler_lstm.transform(val_values_extended.reshape(-1, 1))
    X_val_lstm, _, _, _ = create_sequences(val_values_scaled, seq_len)
    preds_c_val = modelC_p15.predict(X_val_lstm).flatten()
    
    # Hizalama
    min_len = min(len(preds_a_val), len(preds_b_val), len(preds_c_val))
    
    preds_a_val = preds_a_val[:min_len]
    preds_b_val = preds_b_val[:min_len]
    preds_c_val = preds_c_val[:min_len]
    y_val_true_15 = y_val_true_15[:min_len]
    
    # Model D (LightGBM)
    preds_d_val = modelD_p15.predict_proba(X_val_a[:min_len])[:, 1]
    
    # Model E (MLP)
    X_val_mlp = X_val_a.iloc[:min_len][mlp_cols]
    preds_e_val = modelE_p15.predict_proba(X_val_mlp)[:, 1]
    
    # HMM States (Validation kısmı)
    hmm_val = hmm_states_mapped[val_start_idx+1 : val_start_idx+1+min_len]
    
    # 2. Meta-Features Hazırla
    meta_X_val = prepare_meta_features(preds_a_val, preds_b_val, preds_c_val, preds_d_val, preds_e_val, hmm_val)
    
    # 3. Eğit
    meta_model, meta_scaler = train_meta_learner(meta_X_val, y_val_true_15)
    save_meta_learner(meta_model, meta_scaler, output_dir='models')
    
    print("Meta-Learner eğitildi ve kaydedildi.")
    print("Katsayılar (Hangi modele güveniyor?):")
    print(f"A (Cat): {meta_model.coef_[0][0]:.3f}, B (KNN): {meta_model.coef_[0][1]:.3f}, C (LSTM): {meta_model.coef_[0][2]:.3f}")
    print(f"D (LGB): {meta_model.coef_[0][3]:.3f}, E (MLP): {meta_model.coef_[0][4]:.3f}")
else:
    print("Yeterli veri yok.")

In [ ]:
# 10. BÜYÜK FİNAL: Simülasyon ve Karşılaştırmalı Rapor
import matplotlib.pyplot as plt
from jetx_project.simulation import run_simulation
from jetx_project.evaluation import detailed_evaluation
from jetx_project.ensemble import predict_meta

if 'df' in locals() and len(df) > 2000:
    print("--- BÜYÜK FİNAL: Simülasyon Başlıyor ---")
    
    test_size = 2000
    test_df = df.iloc[-test_size:].copy()
    
    # --- Model A (CatBoost) Tahminleri ---
    print("Model A tahmin ediyor...")
    X_test_a, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=len(df)-test_size)
    sim_df = test_df.iloc[:-1].copy()
    
    sim_df['prob_A_1.5'] = modelA_p15.predict_proba(X_test_a)[:, 1]
    sim_df['prob_A_3.0'] = modelA_p3.predict_proba(X_test_a)[:, 1]
    
    # --- Model C (LSTM) Tahminleri ---
    print("Model C (LSTM) tahmin ediyor...")
    seq_len = 200
    raw_values = df['value'].values
    start_idx = len(df) - test_size - seq_len
    test_values_extended = raw_values[start_idx:]
    test_values_scaled = scaler_lstm.transform(test_values_extended.reshape(-1, 1))
    X_lstm, _, _, _ = create_sequences(test_values_scaled, seq_len)
    
    preds_c_p15 = modelC_p15.predict(X_lstm).flatten()
    preds_c_p3 = modelC_p3.predict(X_lstm).flatten()
    
    sim_df['prob_C_1.5'] = preds_c_p15[:-1]
    sim_df['prob_C_3.0'] = preds_c_p3[:-1]
    
    # --- Model D (LightGBM) Tahminleri ---
    print("Model D (LightGBM) tahmin ediyor...")
    sim_df['prob_D_1.5'] = modelD_p15.predict_proba(X_test_a)[:, 1]
    sim_df['prob_D_3.0'] = modelD_p3.predict_proba(X_test_a)[:, 1]
    
    # --- Model E (MLP) Tahminleri ---
    print("Model E (MLP) tahmin ediyor...")
    X_test_mlp = X_test_a[mlp_cols]
    sim_df['prob_E_1.5'] = modelE_p15.predict_proba(X_test_mlp)[:, 1]
    sim_df['prob_E_3.0'] = modelE_p3.predict_proba(X_test_mlp)[:, 1]
    
    # --- Model B (k-NN) Tahminleri (Simülasyon İçin) ---
    print("Model B (k-NN) tahmin ediyor...")
    preds_b_sim = []
    pca_obj = locals().get('pca', None)
    
    for i in range(len(df)-test_size, len(df)-1):
        pat = create_pattern_vector(df['value'].values, i)
        if pat is not None:
            p15, _, _ = predict_model_b(nbrs, pca_obj, targets, pat)
            preds_b_sim.append(p15)
        else:
            preds_b_sim.append(0.5)
    sim_df['prob_B_1.5'] = preds_b_sim
    
    # --- ENSEMBLE (Meta-Learner) Tahminleri ---
    print("Ensemble (Meta-Learner) tahmin ediyor...")
    hmm_sim = hmm_states_mapped[len(df)-test_size+1:]
    
    meta_X_sim = prepare_meta_features(
        sim_df['prob_A_1.5'], 
        sim_df['prob_B_1.5'], 
        sim_df['prob_C_1.5'], 
        sim_df['prob_D_1.5'], 
        sim_df['prob_E_1.5'], 
        hmm_sim
    )
    
    sim_df['prob_Ensemble_1.5'] = predict_meta(meta_model, meta_scaler, meta_X_sim)
    sim_df['prob_Ensemble_3.0'] = sim_df['prob_A_3.0'] 
    
    # --- DETAYLI DEĞERLENDİRME (Yeni Metod) ---
    print("\n--- DETAYLI PERFORMANS ANALİZİ (TP/FP/TN/FN) ---")
    
    y_true_15 = (sim_df['value'] >= 1.5).astype(int)
    y_true_3 = (sim_df['value'] >= 3.0).astype(int)
    
    # Model A
    detailed_evaluation(y_true_15, sim_df['prob_A_1.5'], threshold=0.65, model_name="Model A", target_name="1.5x")
    
    # Ensemble
    detailed_evaluation(y_true_15, sim_df['prob_Ensemble_1.5'], threshold=0.65, model_name="ENSEMBLE (Meta)", target_name="1.5x")
    
    # --- SİMÜLASYON KOŞUSU ---
    
    def run_single_model_sim(name, prob_col_15, prob_col_3):
        temp_df = sim_df.copy()
        temp_df['prob_1.5'] = temp_df[prob_col_15]
        temp_df['prob_3.0'] = temp_df[prob_col_3]
        temp_df['pred_value'] = 0 
        
        print(f"\n>>> {name} Simülasyonu <<<")
        run_simulation(temp_df, model_name=name)
        
    run_single_model_sim("Model A (CatBoost)", 'prob_A_1.5', 'prob_A_3.0')
    run_single_model_sim("ENSEMBLE (Meta-Learner)", 'prob_Ensemble_1.5', 'prob_Ensemble_3.0')
    
    # --- GÜVEN DAĞILIM GRAFİKLERİ ---
    print("\n--- Model Güven Dağılımları Hazırlanıyor ---")
    plt.figure(figsize=(15, 6))
    
    plt.title("1.5x Üstü Olasılık Dağılımı (Modeller Ne Kadar Emin?)")
    plt.hist(sim_df['prob_A_1.5'], bins=50, alpha=0.5, label='Model A (CatBoost)')
    plt.hist(sim_df['prob_Ensemble_1.5'], bins=50, alpha=0.5, label='ENSEMBLE (Meta)')
    plt.axvline(x=0.5, color='k', linestyle='--', label='Kararsızlık Sınırı (0.5)')
    plt.axvline(x=0.65, color='r', linestyle='--', label='Hedef Eşik (0.65)')
    plt.legend()
    plt.show()
    
else:
    print("Simülasyon için yeterli veri yok.")

In [ ]:
# 11. Modelleri İndir
!zip -r models.zip models/
from google.colab import files
files.download('models.zip')